In [ ]:
import re
import pandas as pd

In [ ]:
f = open('group.txt','r',encoding= 'utf-8')
# f = open('nongroup.txt','r',encoding= 'utf-8')

In [ ]:
data = f.read()

In [ ]:
pattern = r'\[(\d{2}/\d{2}/\d{2}, \d{2}:\d{2}:\d{2}\s*[AP]M)\] (.*?)$'

In [ ]:
messages = []

for match in re.findall(pattern, data, re.MULTILINE):
    messages.append(match[1].strip())

In [ ]:
dates = []

for match in re.findall(pattern, data, re.MULTILINE):
    dates.append(match[0].strip())

In [ ]:
df = pd.DataFrame({'message_date':dates,'user_message':messages})
df['message_date'] = pd.to_datetime(df['message_date'], format='%d/%m/%y, %I:%M:%S %p')

df.rename(columns={'message_date': 'date'}, inplace=True)

df.head()

In [ ]:
df.shape

In [ ]:
users = []
messages = []
for message in df['user_message']:
    entry = re.split('([\w\W]+?):\s',message)
    if entry[1:]: # user name
        users.append(entry[1])
        messages.append(entry[2])
    else:
        users.append('group_notification')
        messages.append(entry[0])

df['user'] = users
df['message'] = messages
df.drop(columns=['user_message'], inplace=True)

df.head()

In [ ]:
df['year'] = df['date'].dt.year

In [ ]:
df['month'] = df['date'].dt.month_name()

In [ ]:
df['day'] = df['date'].dt.day

In [ ]:
df['hour'] = df['date'].dt.hour

In [ ]:
df['minute'] = df['date'].dt.minute

In [ ]:
df.head()

In [ ]:
df[df['user'] == '~ Sowmesh Sharma'].shape

In [ ]:
words = []
for message in df['message']:
    words.extend(message.split())

In [ ]:
len(words)

In [ ]:
# df[df['message'] == '\u200Eimage omitted']

In [ ]:
from urlextract import URLExtract

extractor = URLExtract()
urls = extractor.find_urls("Let's www.gamil.com have URL stackoverflow.com as an example google.com, http://facebook.com, ftp://url.in")
urls

In [ ]:
links = []

for message in df['message']:
    links.extend(extractor.find_urls(message))

In [ ]:
len(links)

In [ ]:
x = df['user'].value_counts().head()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
name = x.index
count = x.values

In [ ]:
plt.bar(name,count)
plt.xticks(rotation='vertical')
plt.show()

In [ ]:
round((df['user'].value_counts()/df.shape[0])*100,2).reset_index().rename(columns={'index':'name', 'user':'percent'})

In [ ]:
words = []

for message in df['message']:
    words.extend(message.split())

In [ ]:
from collections import Counter
pd.DataFrame(Counter(words).most_common(20))

In [ ]:
# reomve grp notification
temp = df[df['user'] != 'group_notification']
temp = temp[temp['message'] != '\u200Eimage omitted']

In [ ]:
f = open('hinglish.txt','r')
stop_words = f.read()

In [ ]:
words = []

for message in temp['message']:
        for word in message.lower().split():
            if word not in stop_words:
                words.append(word)

In [ ]:
from collections import Counter
most_common_df = pd.DataFrame(Counter(words).most_common(20))

In [ ]:
plt.barh(most_common_df[0], most_common_df[1])
plt.xticks(rotation='vertical')
plt.show()

In [ ]:
import emoji

emojis = []
for message in df['message']:
    emojis.extend([c for c in message if c in emoji.EMOJI_DATA])

In [ ]:
emoji_df = pd.DataFrame(Counter(emojis).most_common(len(Counter(emojis))))

In [ ]:
plt.bar(emoji_df[1].head(), emoji_df[0].head())
plt.show()

In [ ]:
plt.pie(emoji_df[1].head(),labels=emoji_df[0].head(),autopct="%0.2f")
plt.show()

In [ ]:
df['month_num'] = df["date"].dt.month

In [ ]:
timeline = df.groupby(['year', 'month_num', 'month'])['message'].count().reset_index()

In [ ]:
time = []
for i in range(timeline.shape[0]):
    time.append(timeline['month'][i] + "." + str(timeline['year'][i]))

In [ ]:
timeline['time'] = time

In [ ]:
plt.plot(timeline['time'], timeline['message'])
plt.xticks(rotation='vertical')
plt.show()

In [ ]:
df['only_date'] = df['date'].dt.date

In [ ]:
daily_timeline = df.groupby('only_date').count()['message'].reset_index()

In [ ]:
plt.figure(figsize=(18,10))
plt.plot(daily_timeline['only_date'], daily_timeline['message'])

In [ ]:
df.head()

In [ ]:
df['day_name'] = df['date'].dt.day_name()

In [ ]:
df['day_name'].value_counts()

In [ ]:
df['month'].value_counts()

In [ ]:
df.head()

In [ ]:
period = []

for hour in df[['day_name', 'hour']]['hour']:
    if hour == 23:
        period.append(str(hour) + "-" + str("00"))
    elif hour == 0:
        period.append(str('00') + "-" + str(hour+1))
    else:
        period.append(str(hour) + "-" + str(hour+1))

In [ ]:
df['period'] = period

In [ ]:
df.sample(5)

In [ ]:
import seaborn as sns
plt.figure(figsize=(20,6))
sns.heatmap(df.pivot_table(index='day_name',columns='period',values='message',aggfunc='count').fillna(0))
plt.yticks(rotation='horizontal')
plt.show()